In [102]:
from PIL import Image
import numpy as np
import cv2
import dlib
from pathlib import Path
from tensorflow.keras.utils import get_file
import argparse
from omegaconf import OmegaConf
from src.factory import get_model

In [15]:
def base64_to_PIL_image(base64_img):
    return Image.open(BytesIO(base64.b64decode(base64_img)))

In [16]:
def PIL_image_to_base64(pil_image):
    buffered = BytesIO()
    img1=pil_image.save(buffered, format="JPEG")
    return base64.b64encode(buffered.getvalue())

In [72]:
image = Image.open("./sample_images/images2.jpeg")

In [33]:
pretrained_model = "https://github.com/yu4u/age-gender-estimation/releases/download/v0.6/EfficientNetB3_224_weights.11-3.44.hdf5"

In [35]:
modhash = '6d7f7b7ced093a8b3ef6399163da6ece'
weight_file = get_file("EfficientNetB3_224_weights.11-3.44.hdf5", pretrained_model,
                       cache_subdir="pretrained_models",
                       file_hash=modhash)

132434532/132434532 [==============================] - 126s 1us/step


In [73]:
model_name, img_size = Path(weight_file).stem.split("_")[:2]

In [74]:
img_size

'224'

In [75]:
#image = base64_to_PIL_image(image_string)
img = np.array(image)
input_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
input_img = cv2.flip(input_img, 1)

In [76]:
input_img.shape

(463, 662, 3)

In [77]:
detector = dlib.get_frontal_face_detector()

In [84]:
img_h, img_w, _ = np.shape(input_img)
# detect faces using dlib detector
detected = detector(input_img, 1)
img_size=int(img_size)
faces = np.empty((len(detected),img_size,img_size, 3))

In [85]:
detected

rectangles[[(253, 98) (408, 253)]]

In [86]:
faces.shape

(1, 224, 224, 3)

In [87]:
def get_args():
    parser = argparse.ArgumentParser(description="This script detects faces from web cam input, "
                                                 "and estimates age and gender for the detected faces.",
                                     formatter_class=argparse.ArgumentDefaultsHelpFormatter)
    parser.add_argument("--weight_file", type=str, default=None,
                        help="path to weight file (e.g. weights.28-3.73.hdf5)")
    parser.add_argument("--margin", type=float, default=0.4,
                        help="margin around detected face for age-gender estimation")
    parser.add_argument("--image_dir", type=str, default=None,
                        help="target image directory; if set, images in image_dir are used instead of webcam")
    args = parser.parse_args()
    return args

In [88]:
# args = get_args()
# weight_file = args.weight_file
margin = 0.4

In [89]:
for i, d in enumerate(detected):
    x1, y1, x2, y2, w, h = d.left(), d.top(), d.right() + 1, d.bottom() + 1, d.width(), d.height()
    xw1 = max(int(x1 - margin * w), 0)
    yw1 = max(int(y1 - margin * h), 0)
    xw2 = min(int(x2 + margin * w), img_w - 1)
    yw2 = min(int(y2 + margin * h), img_h - 1)
    print(f"xw1,yw1 {xw1,yw1}, xw2,yw2 {xw2,yw2}")
    cv2.rectangle(input_img, (x1, y1), (x2, y2), (255, 0, 0), 2)
    # cv2.rectangle(img, (xw1, yw1), (xw2, yw2), (255, 0, 0), 2)
    print(input_img.shape)
    faces[i] = cv2.resize(input_img, (img_size, img_size))

xw1,yw1 (190, 35), xw2,yw2 (471, 316)
(463, 662, 3)


In [100]:
cfg = OmegaConf.from_dotlist(["model.model_name=EfficientNetB3", "model.img_size=224"])

In [103]:
model = get_model(cfg)

2024-05-29 12:29:31.224224: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/raghavs/anaconda3/envs/cv_env/lib/python3.7/site-packages/cv2/../../lib64:
2024-05-29 12:29:31.224258: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2024-05-29 12:29:31.224280: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (smartData-442): /proc/driver/nvidia/version does not exist
2024-05-29 12:29:31.224930: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the

In [105]:
faces[0].shape

(224, 224, 3)

In [116]:
results = model.predict(faces)

1/1 [==============================] - 0s 70ms/step


In [117]:
predicted_genders = results[0]

In [118]:
predicted_genders

array([[0.69807833, 0.30192158]], dtype=float32)

In [121]:
ages = np.arange(0, 101).reshape(101, 1)

In [122]:
predicted_ages = results[1].dot(ages).flatten()

In [124]:
results[1].dot(ages)

array([[49.51550296]])